In [3]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow as tf
from pandas import Series
from sklearn.preprocessing import MinMaxScaler

YEAR = 2010


In [16]:
# load targets and convert logs to classification states = ( 1, 0, -1 ) 

y = pd.read_csv(f'../../db/data/secondary/{YEAR}/logs_.csv', index_col=0)
y.reset_index(inplace=True, drop=True)

# converting logs to 3 states = ( 1, 0, -1 ) 
def condition(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0

y_ = y.copy()

for sym in y.columns:
    y_[sym] = y[sym].map( lambda x: condition(x) )

y_ = y_[1:] # shift 1
y_.reset_index(inplace=True, drop=True)
y_


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
0,-1,1,-1,1,-1,1,1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,1,1
1,-1,-1,1,1,1,1,1,1,1,-1,...,1,1,1,-1,-1,-1,1,-1,-1,-1
2,1,1,-1,1,1,-1,-1,-1,-1,1,...,-1,-1,-1,1,-1,-1,-1,1,1,-1
3,1,1,1,-1,1,1,1,-1,-1,-1,...,0,-1,1,1,1,1,1,-1,-1,-1
4,1,-1,1,-1,1,-1,1,1,-1,-1,...,1,-1,1,1,-1,1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6138,-1,-1,-1,0,-1,1,1,1,-1,-1,...,1,1,1,-1,-1,-1,-1,-1,-1,1
6139,-1,-1,1,1,-1,-1,1,1,1,1,...,1,1,1,-1,-1,-1,-1,-1,-1,1
6140,1,-1,1,-1,1,-1,-1,1,-1,1,...,1,-1,1,1,1,1,1,-1,-1,-1
6141,-1,-1,-1,1,-1,-1,-1,-1,1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1


In [6]:
X = pd.read_csv('../../db/data/tertiary/2010/logs_.csv', index_col=0)

# load X features df TODO

X.reset_index(inplace=True, drop=True)
X = X[:-1] # lag -1 x rows

X

,AUD,CAD,CHF,EUR,GBP,JPY,NZD,USD
0,0.061881,-0.084664,0.028045,-0.037279,0.040824,-0.010721,-0.147664,0.149580
1,-0.033417,0.125230,-0.050101,-0.012460,-0.073844,0.025457,-0.096032,0.115168
2,-0.032196,0.252279,0.047114,0.011464,-0.026092,-0.043892,-0.052658,-0.156018
3,0.052931,-0.112991,0.009011,-0.033590,-0.107902,0.156964,0.005096,0.030482
4,0.020095,0.010240,-0.020462,0.002050,-0.016285,-0.014165,0.046656,-0.028128
...,...,...,...,...,...,...,...,...
6138,0.102486,-0.107866,-0.001590,0.039943,-0.054282,0.002682,0.140173,-0.121546
6139,-0.071324,0.108883,0.080836,-0.067406,0.020362,-0.006284,-0.070998,0.005932
6140,-0.020637,-0.002772,-0.002709,0.106780,0.032287,-0.045206,-0.055536,-0.012206
6141,0.016866,-0.030659,0.024334,-0.010662,0.006116,-0.018669,0.075002,-0.062330


In [17]:
# scaling features

def min_max ( data ):

    series = Series(data)
    print(series)
    # prepare data for normalization
    values = series.values
    values = values.reshape((len(values), 1))
    # train the normalization
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(values)
    print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
    # normalize the dataset and print
    normalized = scaler.transform(values)
    print(normalized)
    # inverse transform and print
    inversed = scaler.inverse_transform(normalized)
    print(inversed)

    return normalized, inversed


X_ = []

for x in X.columns:
    normalized, _ = min_max( X[x] )
    X_.append(normalized)

X_ = np.array(X_)

X_


0       0.061881
1      -0.033417
2      -0.032196
3       0.052931
4       0.020095
          ...   
6138    0.102486
6139   -0.071324
6140   -0.020637
6141    0.016866
6142   -0.012995
Name: AUD, Length: 6143, dtype: float64
Min: -0.904309, Max: 0.796319
[[0.13627379]
 [0.02420048]
 [0.02563605]
 ...
 [0.03922963]
 [0.08333521]
 [0.04821681]]
[[ 0.06188058]
 [-0.03341689]
 [-0.03219621]
 ...
 [-0.0206374 ]
 [ 0.01686618]
 [-0.01299547]]
0      -0.084664
1       0.125230
2       0.252279
3      -0.112991
4       0.010240
          ...   
6138   -0.107866
6139    0.108883
6140   -0.002772
6141   -0.030659
6142    0.016503
Name: CAD, Length: 6143, dtype: float64
Min: -1.296866, Max: 1.101955
[[0.01066412]
 [0.18566176]
 [0.29158815]
 ...
 [0.07894114]
 [0.05569111]
 [0.09501134]]
[[-0.08466443]
 [ 0.12522957]
 [ 0.25227878]
 ...
 [-0.00277225]
 [-0.03065858]
 [ 0.01650251]]
0       0.028045
1      -0.050101
2       0.047114
3       0.009011
4      -0.020462
          ...   
6138   -0.00

array([[[ 0.13627379],
        [ 0.02420048],
        [ 0.02563605],
        ...,
        [ 0.03922963],
        [ 0.08333521],
        [ 0.04821681]],

       [[ 0.01066412],
        [ 0.18566176],
        [ 0.29158815],
        ...,
        [ 0.07894114],
        [ 0.05569111],
        [ 0.09501134]],

       [[ 0.08580264],
        [-0.00398903],
        [ 0.10771381],
        ...,
        [ 0.05046553],
        [ 0.08153966],
        [ 0.07158959]],

       ...,

       [[-0.3424794 ],
        [-0.32739089],
        [-0.35631398],
        ...,
        [-0.35686184],
        [-0.34579417],
        [-0.32006913]],

       [[-0.00872045],
        [ 0.04530313],
        [ 0.09068594],
        ...,
        [ 0.08767424],
        [ 0.22425891],
        [ 0.0634114 ]],

       [[ 0.27188529],
        [ 0.21778411],
        [-0.20856724],
        ...,
        [ 0.01752966],
        [-0.061273  ],
        [ 0.05264711]]])

In [18]:

train = 0.7
valid = 0.9

x_n = len(X)
y_n = len(y)

X__, y__ = {}, {}

X__['train'] = X_[0:int(x_n*train)]
X__['valid'] = X_[int(x_n*train):int(x_n*valid)]
X__['test']  = X_[int(x_n*valid):]
y__['train'] = y_[0:int(y_n*train)]
y__['valid'] = y_[int(y_n*train):int(y_n*valid)]
y__['test']  = y_[int(y_n*valid):]

num_features = X.shape[1]

num_features

X_.shape

(8, 6143, 1)

In [19]:
n_lag = 1
n_neurons = 200
n_epochs = 1000
n_batch = 1
n_neurons = 1

# design network
model = Sequential()

model.add(LSTM(n_neurons, stateful=True))

model.add(Dense(y.shape[1]))

model.compile(optimizer='adam', loss='mse')

In [20]:

MAX_EPOCHS = 10000

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.Accuracy()])

  history = model.fit(X=X__['train'], y=y__['train'], epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  
  return history


In [21]:
compile_and_fit(model, 

SyntaxError: unexpected EOF while parsing (<ipython-input-21-b065d77e4dba>, line 1)

In [61]:
model.fit(x=X_['train'], y=y_['train'])

ValueError: in user code:

    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 214, in assert_input_compatibility
        ra